# ReAct Agent with Chat History

In [ ]:
from langchain.llms.huggingface_endpoint import HuggingFaceEndpoint
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import os

HF_API_TOKEN = os.environ['HUGGINGFACE_API_TOKEN']

llm = HuggingFaceEndpoint(repo_id="meta-llama/Meta-Llama-3-70B-Instruct", 
                             temperature=0.001,
                             streaming=True,
                             callbacks=[StreamingStdOutCallbackHandler()],
                             huggingfacehub_api_token=HF_API_TOKEN)

/home/geekmdtravis/.local/share/virtualenvs/langchain-learning-KI_9i9mN/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/geekmdtravis/.cache/huggingface/token
Login successful


## Create the Custom Tool

In [82]:
from typing import List, Dict
def make_panel_object(synonyms: List[int], desc: str, oid: int):
    return {
            "synonyms": synonyms,
            "description": desc,
            "oid": oid
    }
tool_map: Dict[str, Dict[List[str], int]] = {
    "basic metabolic panel": make_panel_object(["bmp", "renal function"], "A Basic Metabolic Panel is a standard diagnostic test that evaluates various essential components of the body's metabolism. Typically performed through a blood sample, it assesses electrolyte levels, kidney function, and blood glucose levels. The panel usually includes tests for electrolytes like sodium, potassium, and chloride, as well as measures of kidney function such as blood urea nitrogen (BUN) and creatinine. Glucose levels are also included to screen for diabetes or monitor blood sugar control.", 1),
    "comprehensive metabolic panel": make_panel_object(["cmp", "liver function", "lft", "liver function test", "liver function", "liver panel"], "A Comprehensive Metabolic Panel is a more extensive version of the Basic Metabolic Panel, providing additional information about liver function and protein levels. In addition to the tests included in a BMP, a CMP typically includes measurements of liver enzymes like alanine aminotransferase (ALT) and aspartate aminotransferase (AST), as well as total protein and albumin levels. The panel helps assess overall metabolic health and can detect conditions such as liver disease, kidney dysfunction, and diabetes.", 2),
    "complete blood count": make_panel_object(["cbc", "blood count"], "A Complete Blood Count is a common blood test that provides important information about the types and numbers of cells in the blood. It includes measurements of red blood cells, white blood cells, and platelets, as well as other parameters like hemoglobin and hematocrit. The test can help diagnose various conditions, such as anemia, infections, and blood disorders.", 3),
    "lipid panel": make_panel_object(["cholesterol", "triglycerides"], "A Lipid Panel is a blood test that measures the levels of fats and cholesterol in the blood. It typically includes measurements of total cholesterol, low-density lipoprotein (LDL) cholesterol, high-density lipoprotein (HDL) cholesterol, and triglycerides. The test helps assess the risk of heart disease and provides information about an individual's lipid profile.", 4),
    "thyroid panel": make_panel_object(["thyroid function", "tsh"], "A Thyroid Panel is a group of blood tests that evaluate the function of the thyroid gland. It typically includes measurements of thyroid-stimulating hormone (TSH), thyroxine (T4), and triiodothyronine (T3) levels. The panel helps diagnose thyroid disorders such as hypothyroidism and hyperthyroidism, as well as monitor treatment for these conditions.", 5),
    "hemoglobin a1c%": make_panel_object(["a1c", "glycated hemoglobin"], "Hemoglobin A1c (HbA1c) is a blood test that measures the average blood sugar levels over the past 2-3 months. It is used to monitor long-term glucose control in individuals with diabetes and assess the risk of complications. The test provides valuable information about blood sugar management and helps guide treatment decisions.",6),
}
tool_map


{'basic metabolic panel': {'synonyms': ['bmp', 'renal function'],
  'description': "A Basic Metabolic Panel is a standard diagnostic test that evaluates various essential components of the body's metabolism. Typically performed through a blood sample, it assesses electrolyte levels, kidney function, and blood glucose levels. The panel usually includes tests for electrolytes like sodium, potassium, and chloride, as well as measures of kidney function such as blood urea nitrogen (BUN) and creatinine. Glucose levels are also included to screen for diabetes or monitor blood sugar control.",
  'oid': 1},
 'comprehensive metabolic panel': {'synonyms': ['cmp',
   'liver function',
   'lft',
   'liver function test',
   'liver function',
   'liver panel'],
  'description': 'A Comprehensive Metabolic Panel is a more extensive version of the Basic Metabolic Panel, providing additional information about liver function and protein levels. In addition to the tests included in a BMP, a CMP typically

In [87]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def get_order_id(panel_name: str) -> int:
    """Get an order ID for a given panel by name. When you 
    feed a panel name to this tool, it will return the order ID. 
    """
    clean_panel_name = panel_name.lower().split("\n")[0].strip()
    # map a synonmym to a panel name
    keys = tool_map.keys()
    for key in keys:
        if clean_panel_name in tool_map[key]["synonyms"]:
            clean_panel_name = key
            break
    try:
        return tool_map[clean_panel_name]["oid"]
    except KeyError:
        return -1

tools = [get_order_id]

get_order_id("Comprehensive Metabolic Panel")

2

## Create the ReAct Agent

In [84]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")

# Construct the ReAct agent
agent = create_react_agent(llm, [get_order_id], prompt)

# Run the Agent

In [92]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=[get_order_id], verbose=True)
query = "Which lab panel trends diabetes levels? whatever that panel is, give me the order ID for it."
res = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
 I need to figure out which panel trends diabetes levels. I know that HbA1c is a common test for diabetes, so I'll try that.
Action: get_order_id
Action Input: HbA1c
Observation I need to figure out which panel trends diabetes levels. I know that HbA1c is a common test for diabetes, so I'll try that.
Action: get_order_id
Action Input: HbA1c
Observation-1 That didn't work. Maybe it's under a different name. I know that HbA1c is also known as Hemoglobin A1c, so I'll try that.
Action: get_order_id
Action Input: Hemoglobin A1c
Observation That didn't work. Maybe it's under a different name. I know that HbA1c is also known as Hemoglobin A1c, so I'll try that.
Action: get_order_id
Action Input: Hemoglobin A1c
Observation-1 That didn't work either. Maybe it's under a different name. I know that HbA1c is also known as Glycohemoglobin, so I'll try that.
Action: get_order_id
Action Input: Glycohemoglobin
Observation That didn't work either. Maybe it's unde